In [1]:
from requests import get as get_request
from io import BytesIO
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import altair as alt

In [5]:

filename = 'Building_Permits_20251125.csv'
permits = pd.read_csv(os.path.join(os.getcwd(), 'data', filename))

C:\Users\ellin\AppData\Local\Temp\ipykernel_27480\2644830692.py:2: DtypeWarning: Columns (25,26,27,28,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  permits = pd.read_csv(os.path.join(os.getcwd(), 'data', filename))


In [6]:
print(permits.shape)
print(permits.isna().sum(axis=0))

(185867, 40)
PermitNum                         0
PermitClass                    6820
PermitClassMapped              6820
PermitTypeMapped                  0
PermitTypeDesc                12264
Description                     674
HousingUnits                      0
HousingUnitsRemoved           97586
HousingUnitsAdded             97586
EstProjectCost                35112
AppliedDate                   43895
IssuedDate                    50778
ExpiresDate                   50644
CompletedDate                 82893
StatusCurrent                     0
RelatedMup                   171608
OriginalAddress1                856
OriginalCity                   8290
OriginalState                  8161
OriginalZip                    9361
ContractorCompanyName        156105
Link                              0
Latitude                       1005
Longitude                      1005
Location1                      1005
TotalDaysPlanReview          156452
DaysInitialPlanReview        156047
DaysPlanReviewC

In [7]:
permits.head()

,PermitNum,PermitClass,PermitClassMapped,PermitTypeMapped,PermitTypeDesc,Description,HousingUnits,HousingUnitsRemoved,HousingUnitsAdded,EstProjectCost,...,InitialReviewCompleteDate,PlanReviewCompleteDate,DaysIssuePermitCity,ReadyToIssueDate,Zoning,DwellingUnitType,StandardPlan,DependentBuilding,ParentPermitNum,HousingCategory
0,3001776-EX,Commercial,Non-Residential,ECA and Shoreline Exemption/Street Improvement...,Environmentally Critical Area Exemption,Exception/Exemption Request for: Land Use Appl...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,3002256-EX,Single Family/Duplex,Residential,ECA and Shoreline Exemption/Street Improvement...,Environmentally Critical Area Exemption,Exception/Exemption Request for: CANCELLED 3/8...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,3002287-EX,Single Family/Duplex,Residential,ECA and Shoreline Exemption/Street Improvement...,Environmentally Critical Area Exemption,Exception/Exemption Request for: Land Use Appl...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
3,3003446-EX,Industrial,Non-Residential,ECA and Shoreline Exemption/Street Improvement...,Shoreline Exemption,Exception/Exemption Request for: CANCELLED 3/2...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,3003449-EX,Multifamily,Residential,ECA and Shoreline Exemption/Street Improvement...,Environmentally Critical Area Exemption,Exception/Exemption Request for: CANCELED PER ...,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [9]:
permits.Description

0         Exception/Exemption Request for: Land Use Appl...
1         Exception/Exemption Request for: CANCELLED 3/8...
2         Exception/Exemption Request for: Land Use Appl...
3         Exception/Exemption Request for: CANCELLED 3/2...
4         Exception/Exemption Request for: CANCELED PER ...
                                ...                        
185862    Construct engineered voluntary seismic retrofi...
185863    Central OMF Site Improvements and site modific...
185864    Construct east one-family dwelling  per plan. ...
185865    Construct non-structural repairs to condominiu...
185866    Construct commercial floating building (Boat S...
Name: Description, Length: 185867, dtype: object

### Filter for Earthquake Repair and Seismic Retrofit permits

In [12]:
RETROFIT_PHRASES = [
    'seismic retrofit',
    'seismic upgrade',
    'seismic proof',
    'seismic home retrofit',
    'seismic home upgrade',
    'seismic home proof',
    'seismically retrofit',
    'seismically upgrade',
    'seismically proof',
    'earthquake retrofit',
    'earthquake upgrade',
    'earthquake proof',
    'earthquake home retrofit',
    'earthquake home upgrade',
    'earthquake home proof'
]

def check_for_retrofits(description, check_phrases = RETROFIT_PHRASES):
    '''
    Checks description for mentions of seismic retrofits and earthquake retrofits
    '''
    if pd.isna(description):
        return False
    normalized_description = ' '.join(description.split()).lower()
    return any([phrase in normalized_description for phrase in check_phrases])
    

retrofits = permits[permits['Description'].apply(func=check_for_retrofits)]
retrofits.shape

(4516, 40)

In [13]:
retrofits.head()

,PermitNum,PermitClass,PermitClassMapped,PermitTypeMapped,PermitTypeDesc,Description,HousingUnits,HousingUnitsRemoved,HousingUnitsAdded,EstProjectCost,...,InitialReviewCompleteDate,PlanReviewCompleteDate,DaysIssuePermitCity,ReadyToIssueDate,Zoning,DwellingUnitType,StandardPlan,DependentBuilding,ParentPermitNum,HousingCategory
678,6062875-CN,Single Family/Duplex,Residential,Building,Addition/Alteration,Volutary seismic upgrade to existing single-fa...,0,0.0,0.0,"4,300",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Single-Family Add/Alt
859,6063832-CN,Commercial,Non-Residential,Building,Addition/Alteration,Voluntary Seismic Retrofit at roof level only ...,0,0.0,0.0,"208,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Commercial Add/Alt
1040,6064791-CN,Single Family/Duplex,Residential,Building,Addition/Alteration,Closed as Incomplete - Expired Permit REMOV...,0,NaN,NaN,"17,760",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Single-Family Add/Alt
1090,6065241-CN,Single Family/Duplex,Residential,Building,Addition/Alteration,Construct dormer addition per plan and seismic...,0,0.0,0.0,"50,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Single-Family Add/Alt
1214,6066366-CN,Single Family/Duplex,Residential,Building,Addition/Alteration,seismic retrofit for project impact,0,0.0,0.0,"3,000",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,Single-Family Add/Alt
